# 3.4.1 基本面分析

In [1]:
# 使用 tushare 获取股票基本面数据 - 添加安全函数
import os
import tushare as ts

# 从环境变量获取 token
token = os.getenv('TUSHARE_TOKEN')
if not token:
    raise ValueError("请设置环境变量 TUSHARE_TOKEN")

# 初始化 tushare
ts.set_token(token)
pro = ts.pro_api()

# 安全的列访问函数定义
def safe_get_column(df, col_names, default=None):
    """安全地获取列值"""
    for col in col_names:
        if col in df.columns:
            return df[col].iloc[0]
    print(f"⚠️ 警告: 未找到列 {col_names}")
    return default

# 获取美的集团的财务数据（示例使用美的集团 000333 作为替代，因为微软不在 tushare 数据库中）
# 这里是示例，您可以根据需要替换为其他股票代码
df_income = pro.income(ts_code='000333.SZ', period='20231231')  # 2023年年报
print("损益表形状:", df_income.shape)
print("前5列:", list(df_income.columns[:5]))
df_income

损益表形状: (1, 85)
前5列: ['ts_code', 'ann_date', 'f_ann_date', 'end_date', 'report_type']


,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,end_type,basic_eps,diluted_eps,total_revenue,...,withdra_biz_devfund,withdra_rese_fund,withdra_oth_ersu,workers_welfare,distr_profit_shrhder,prfshare_payable_dvd,comshare_payable_dvd,capit_comstock_div,continued_net_profit,update_flag
0,000333.SZ,20240328,20240328,20231231,1,1,4,4.93,4.92,3.737098e+11,...,None,None,None,None,None,None,None,None,3.374535e+10,1


In [2]:
# 获取资产负债表数据
df_balance = pro.balancesheet(ts_code='000333.SZ', period='20231231')  # 2023年年报
df_balance

,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,end_type,total_share,cap_rese,undistr_porfit,...,accounts_receiv_bill,accounts_pay,oth_rcv_total,fix_assets_total,cip_total,oth_pay_total,long_pay_total,debt_invest,oth_debt_invest,update_flag
0,000333.SZ,20240328,20240328,20231231,1,1,4,7.025769e+09,2.124316e+10,1.362843e+11,...,3.840670e+10,9.423807e+10,2.181878e+09,3.093796e+10,4.681220e+09,4.442928e+09,None,None,6.319047e+09,0
1,000333.SZ,20240328,20240328,20231231,1,1,4,7.025769e+09,2.124316e+10,1.362843e+11,...,3.840670e+10,9.423807e+10,2.181878e+09,3.093796e+10,4.681220e+09,4.442928e+09,None,None,6.319047e+09,1


In [3]:
# 获取现金流量表数据
df_cashflow = pro.cashflow(ts_code='000333.SZ', period='20231231')  # 2023年年报
df_cashflow

,ts_code,ann_date,f_ann_date,end_date,comp_type,report_type,end_type,net_profit,finan_exp,c_fr_sale_sg,...,net_dism_capital_add,net_cash_rece_sec,credit_impa_loss,use_right_asset_dep,oth_loss_asset,end_bal_cash,beg_bal_cash,end_bal_cash_equ,beg_bal_cash_equ,update_flag
0,000333.SZ,20240328,20240328,20231231,1,1,4,3.374535e+10,-3.134637e+09,3.560760e+11,...,None,None,198624000.0,None,None,5.988726e+10,5.113197e+10,None,None,1


In [4]:
# 计算财务比率示例（EBITDA）- 修复版
import pandas as pd

# 安全的列访问函数
def safe_get_column(df, col_names, default=None):
    """安全地获取列值"""
    for col in col_names:
        if col in df.columns:
            return df[col].iloc[0]
    print(f"⚠️ 警告: 未找到列 {col_names}")
    return default

# 首先检查实际的列名
print("=== 调试信息 ===")
print("损益表所有列名:", list(df_income.columns))
profit_cols = [col for col in df_income.columns if 'profit' in col.lower()]
print("\n可能的净利润相关列:", profit_cols)
revenue_cols = [col for col in df_income.columns if 'revenue' in col.lower() or 'income' in col.lower()]
print("可能的收入相关列:", revenue_cols)
print("================")

# 计算净利润率 - 使用正确的列名
net_profit = safe_get_column(df_income, ['continued_net_profit', 'net_profit', 'total_profit'])
total_revenue = safe_get_column(df_income, ['total_revenue', 'biz_income', 'revenue'])

if net_profit is not None and total_revenue is not None and total_revenue != 0:
    profit_margin = (net_profit / total_revenue * 100).round(2)
    print(f"\n✅ 净利润率: {profit_margin}%")
else:
    print("\n❌ 无法计算净利润率")

=== 调试信息 ===
损益表所有列名: ['ts_code', 'ann_date', 'f_ann_date', 'end_date', 'report_type', 'comp_type', 'end_type', 'basic_eps', 'diluted_eps', 'total_revenue', 'revenue', 'int_income', 'prem_earned', 'comm_income', 'n_commis_income', 'n_oth_income', 'n_oth_b_income', 'prem_income', 'out_prem', 'une_prem_reser', 'reins_income', 'n_sec_tb_income', 'n_sec_uw_income', 'n_asset_mg_income', 'oth_b_income', 'fv_value_chg_gain', 'invest_income', 'ass_invest_income', 'forex_gain', 'total_cogs', 'oper_cost', 'int_exp', 'comm_exp', 'biz_tax_surchg', 'sell_exp', 'admin_exp', 'fin_exp', 'assets_impair_loss', 'prem_refund', 'compens_payout', 'reser_insur_liab', 'div_payt', 'reins_exp', 'oper_exp', 'compens_payout_refu', 'insur_reser_refu', 'reins_cost_refund', 'other_bus_cost', 'operate_profit', 'non_oper_income', 'non_oper_exp', 'nca_disploss', 'total_profit', 'income_tax', 'n_income', 'n_income_attr_p', 'minority_gain', 'oth_compr_income', 't_compr_income', 'compr_inc_attr_p', 'compr_inc_attr_m_s', '

In [5]:
# 获取季度财务报表
# tushare 可以获取指定年度的季度数据
df_income_quarterly = pro.income(ts_code='000333.SZ', period='20231231')  # 获取季度数据需要使用不同参数
df_income_quarterly

,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,end_type,basic_eps,diluted_eps,total_revenue,...,withdra_biz_devfund,withdra_rese_fund,withdra_oth_ersu,workers_welfare,distr_profit_shrhder,prfshare_payable_dvd,comshare_payable_dvd,capit_comstock_div,continued_net_profit,update_flag
0,000333.SZ,20240328,20240328,20231231,1,1,4,4.93,4.92,3.737098e+11,...,None,None,None,None,None,None,None,None,3.374535e+10,1


In [6]:
# 获取季度资产负债表
df_balance_quarterly = pro.balancesheet(ts_code='000333.SZ', period='20231231')  # 季度数据
df_balance_quarterly

,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,end_type,total_share,cap_rese,undistr_porfit,...,accounts_receiv_bill,accounts_pay,oth_rcv_total,fix_assets_total,cip_total,oth_pay_total,long_pay_total,debt_invest,oth_debt_invest,update_flag
0,000333.SZ,20240328,20240328,20231231,1,1,4,7.025769e+09,2.124316e+10,1.362843e+11,...,3.840670e+10,9.423807e+10,2.181878e+09,3.093796e+10,4.681220e+09,4.442928e+09,None,None,6.319047e+09,0
1,000333.SZ,20240328,20240328,20231231,1,1,4,7.025769e+09,2.124316e+10,1.362843e+11,...,3.840670e+10,9.423807e+10,2.181878e+09,3.093796e+10,4.681220e+09,4.442928e+09,None,None,6.319047e+09,1


In [7]:
# 获取季度现金流量表
df_cashflow_quarterly = pro.cashflow(ts_code='000333.SZ', period='20231231')  # 季度数据
df_cashflow_quarterly

,ts_code,ann_date,f_ann_date,end_date,comp_type,report_type,end_type,net_profit,finan_exp,c_fr_sale_sg,...,net_dism_capital_add,net_cash_rece_sec,credit_impa_loss,use_right_asset_dep,oth_loss_asset,end_bal_cash,beg_bal_cash,end_bal_cash_equ,beg_bal_cash_equ,update_flag
0,000333.SZ,20240328,20240328,20231231,1,1,4,3.374535e+10,-3.134637e+09,3.560760e+11,...,None,None,198624000.0,None,None,5.988726e+10,5.113197e+10,None,None,1


In [8]:
# 获取股票基本信息
import pandas as pd

# 使用 tushare 获取股票基本信息
stock_basic = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
stock_info = stock_basic[stock_basic['ts_code'] == '000333.SZ'].iloc[0]
stock_info

ts_code      000333.SZ
symbol          000333
name              美的集团
area                广东
industry          家用电器
list_date     20130918
Name: 63, dtype: object

In [9]:
# 显示股票信息字段
print(', '.join(stock_info.keys()))

ts_code, symbol, name, area, industry, list_date


In [10]:
# 获取当前股价（示例）
# 注意：tushare 提供的是历史数据，需要通过其他方式获取实时数据
# 这里只是示例
print(f"美的集团当前价格：{stock_info['name']}")  # 示例显示股票名称

美的集团当前价格：美的集团


In [11]:
# 收集多只股票的基本信息
def collect_stock_info(tickers: list, fields: list):
    """
    使用 tushare 收集多只股票的基本信息
    
    Args:
        tickers: 股票代码列表 (如 ['000333.SZ', '000001.SZ'])
        fields: 要获取的字段列表
    
    Returns:
        pandas DataFrame
    """
    rows = []
    
    # 先获取所有股票的基本信息
    all_stocks = pro.stock_basic(exchange='', list_status='L', 
                                  fields='ts_code,symbol,name,area,industry,market,list_date')
    
    for ticker in tickers:
        # 获取股票基本信息
        stock_row = all_stocks[all_stocks['ts_code'] == ticker]
        if not stock_row.empty:
            row = [ticker] + [stock_row[field].iloc[0] if field in stock_row.columns else None 
                            for field in fields]
            rows.append(row)
    
    return pd.DataFrame(rows, columns=["Ticker"] + fields)

# 示例：获取几只股票的信息
collect_stock_info(['000333.SZ', '000001.SZ', '000002.SZ'], 
                   ['name', 'area', 'industry'])

,Ticker,name,area,industry
0,000333.SZ,美的集团,广东,家用电器
1,000001.SZ,平安银行,深圳,银行
2,000002.SZ,万科A,深圳,全国地产


In [12]:
# 使用函数获取股票信息
collect_stock_info(['000333.SZ', '000001.SZ', '000002.SZ'], ['name', 'area', 'industry'])

,Ticker,name,area,industry
0,000333.SZ,美的集团,广东,家用电器
1,000001.SZ,平安银行,深圳,银行
2,000002.SZ,万科A,深圳,全国地产


In [13]:
# 使用 tushare 获取财务数据 - 修复版
company = '000333.SZ'  # 美的集团
df_income = pro.income(ts_code=company, period='20231231')
df_balance = pro.balancesheet(ts_code=company, period='20231231')

print("损益表形状:", df_income.shape)
print("资产负债表形状:", df_balance.shape)

# 获取最新的报表列 - 使用正确的列名 'end_date'
if not df_income.empty:
    print("\n损益表列名:", list(df_income.columns))
    latest_inc_period = df_income['end_date'].iloc[0]  # 使用 end_date 列（实际存在的列）
    print(f"✅ 最新损益表报告期: {latest_inc_period}")

if not df_balance.empty:
    latest_bs_period = df_balance['end_date'].iloc[0]  # 使用 end_date 列（实际存在的列）
    print(f"✅ 最新资产负债表报告期: {latest_bs_period}")

损益表形状: (1, 85)
资产负债表形状: (2, 152)

损益表列名: ['ts_code', 'ann_date', 'f_ann_date', 'end_date', 'report_type', 'comp_type', 'end_type', 'basic_eps', 'diluted_eps', 'total_revenue', 'revenue', 'int_income', 'prem_earned', 'comm_income', 'n_commis_income', 'n_oth_income', 'n_oth_b_income', 'prem_income', 'out_prem', 'une_prem_reser', 'reins_income', 'n_sec_tb_income', 'n_sec_uw_income', 'n_asset_mg_income', 'oth_b_income', 'fv_value_chg_gain', 'invest_income', 'ass_invest_income', 'forex_gain', 'total_cogs', 'oper_cost', 'int_exp', 'comm_exp', 'biz_tax_surchg', 'sell_exp', 'admin_exp', 'fin_exp', 'assets_impair_loss', 'prem_refund', 'compens_payout', 'reser_insur_liab', 'div_payt', 'reins_exp', 'oper_exp', 'compens_payout_refu', 'insur_reser_refu', 'reins_cost_refund', 'other_bus_cost', 'operate_profit', 'non_oper_income', 'non_oper_exp', 'nca_disploss', 'total_profit', 'income_tax', 'n_income', 'n_income_attr_p', 'minority_gain', 'oth_compr_income', 't_compr_income', 'compr_inc_attr_p', 'com

In [14]:
# 计算 ROA (总资产收益率) - 修复版
# ROA = 净利润 / 总资产 * 100

# 使用安全函数获取列值
net_profit = safe_get_column(df_income, ['continued_net_profit', 'net_profit', 'total_profit'])
total_assets = safe_get_column(df_balance, ['total_assets', 'assets_total'])

if net_profit is not None and total_assets is not None and total_assets != 0:
    roa = (net_profit / total_assets * 100).round(2)
    print(f"✅ ROA (总资产收益率): {roa}%")
else:
    print("❌ 无法计算 ROA")

✅ ROA (总资产收益率): 6.94%


In [15]:
# 计算 ROE (净资产收益率) - 修复版
# ROE = 净利润 / 股东权益 * 100

# 使用安全函数获取列值
net_profit = safe_get_column(df_income, ['continued_net_profit', 'net_profit', 'total_profit'])
total_stockholders_equity = safe_get_column(df_balance, ['total_hldr_eq', 'hldr_eq', 'stockholders_equity'])

if net_profit is not None and total_stockholders_equity is not None and total_stockholders_equity != 0:
    roe = (net_profit / total_stockholders_equity * 100).round(2)
    print(f"✅ ROE (净资产收益率): {roe}%")
else:
    print("❌ 无法计算 ROE")

⚠️ 警告: 未找到列 ['total_hldr_eq', 'hldr_eq', 'stockholders_equity']
❌ 无法计算 ROE


In [16]:
# 计算利润率 - 修复版
# 利润率 = 净利润 / 营业总收入 * 100

# 使用安全函数获取列值
net_profit = safe_get_column(df_income, ['continued_net_profit', 'net_profit', 'total_profit'])
total_revenue = safe_get_column(df_income, ['total_revenue', 'biz_income', 'revenue'])

if net_profit is not None and total_revenue is not None and total_revenue != 0:
    profit_margin = (net_profit / total_revenue * 100).round(2)
    print(f"✅ 利润率: {profit_margin}%")
else:
    print("❌ 无法计算利润率")

✅ 利润率: 9.03%


In [17]:
# 计算资产周转率 - 修复版
# 资产周转率 = 营业总收入 / 总资产

# 使用安全函数获取列值
total_revenue = safe_get_column(df_income, ['total_revenue', 'biz_income', 'revenue'])
total_assets = safe_get_column(df_balance, ['total_assets', 'assets_total'])

if total_revenue is not None and total_assets is not None and total_assets != 0:
    asset_turnover = (total_revenue / total_assets).round(4)
    print(f"✅ 资产周转率: {asset_turnover}")
else:
    print("❌ 无法计算资产周转率")

✅ 资产周转率: 0.7689


In [18]:
# 计算资产负债率 - 修复版
# 资产负债率 = 总负债 / 股东权益

# 使用安全函数获取列值
total_liab = safe_get_column(df_balance, ['total_liab', 'liab_total'])
total_hldr_eq = safe_get_column(df_balance, ['total_hldr_eq', 'hldr_eq'])

if total_liab is not None and total_hldr_eq is not None and total_hldr_eq != 0:
    debt_to_equity = (total_liab / total_hldr_eq).round(4)
    print(f"✅ 资产负债率: {debt_to_equity}")
else:
    print("❌ 无法计算资产负债率")

⚠️ 警告: 未找到列 ['total_hldr_eq', 'hldr_eq']
❌ 无法计算资产负债率


In [19]:
# 多年度资产负债率对比分析 - 修复版
# 获取多年的财务数据进行比较
company_code = '000333.SZ'

# 获取不同年度的资产负债表数据
periods = ['20231231', '20221231', '20211231', '20201231']
debt_ratios = []

for period in periods:
    try:
        df_bs = pro.balancesheet(ts_code=company_code, period=period)
        if not df_bs.empty:
            # 使用安全函数获取列值
            total_liab = safe_get_column(df_bs, ['total_liab', 'liab_total'])
            total_hldr_eq = safe_get_column(df_bs, ['total_hldr_eq', 'hldr_eq'])
            
            if total_liab is not None and total_hldr_eq is not None and total_hldr_eq != 0:
                ratio = (total_liab / total_hldr_eq).round(4)
                debt_ratios.append((period, ratio))
                print(f"✅ {period}: {ratio}")
            else:
                print(f"⚠️  {period}: 数据不完整")
        else:
            print(f"⚠️  {period}: 数据为空")
    except Exception as e:
        print(f"❌ 获取 {period} 数据时出错: {e}")
        continue

print(f"\n成功获取 {len(debt_ratios)} 年的数据")

⚠️ 警告: 未找到列 ['total_hldr_eq', 'hldr_eq']
⚠️  20231231: 数据不完整
⚠️ 警告: 未找到列 ['total_hldr_eq', 'hldr_eq']
⚠️  20221231: 数据不完整
⚠️ 警告: 未找到列 ['total_hldr_eq', 'hldr_eq']
⚠️  20211231: 数据不完整
⚠️ 警告: 未找到列 ['total_hldr_eq', 'hldr_eq']
⚠️  20201231: 数据不完整

成功获取 0 年的数据


# 技术分析 - 字体设置和诊断
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib

# ==================== 字体设置 ====================
def setup_chinese_font():
    """
    设置中文字体，解决中文显示问题
    """
    # 尝试多种中文字体
    chinese_fonts = [
        'SimHei',           # 黑体 (Windows)
        'Arial Unicode MS', # Arial Unicode (macOS)
        'Microsoft YaHei',  # 微软雅黑 (Windows)
        'PingFang SC',      # 苹方 (macOS)
        'Hiragino Sans GB', # 冬青黑体 (macOS)
        'WenQuanYi Zen Hei', # 文泉驿正黑
        'WenQuanYi Micro Hei', # 文泉驿微米黑
    ]
    
    # 检查系统中哪些字体可用
    available_fonts = [f.name for f in matplotlib.font_manager.fontManager.ttflist]
    available_chinese_fonts = [font for font in chinese_fonts if font in available_fonts]
    
    print("🔍 字体检查结果:")
    print(f"系统中可用字体总数: {len(available_fonts)}")
    print(f"可用中文字体: {available_chinese_fonts if available_chinese_fonts else '未找到系统中文字体'}")
    
    # 设置字体（使用找到的第一个中文字体）
    if available_chinese_fonts:
        selected_font = available_chinese_fonts[0]
        plt.rcParams['font.sans-serif'] = [selected_font] + chinese_fonts
        print(f"✅ 已设置字体: {selected_font}")
    else:
        # 如果没有中文字体，使用英文
        plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
        print("⚠️  未找到中文字体，使用默认字体（中文可能显示异常）")
    
    # 设置负号显示
    plt.rcParams['axes.unicode_minus'] = False
    
    return available_chinese_fonts

# 执行字体设置
available_fonts = setup_chinese_font()

# ==================== 字体测试 ====================
def test_chinese_font():
    """
    测试中文字体显示
    """
    plt.figure(figsize=(8, 4))
    plt.text(0.5, 0.7, "中文字体测试", fontsize=16, ha='center')
    plt.text(0.5, 0.5, "Chinese Font Test", fontsize=14, ha='center')
    plt.text(0.5, 0.3, "涨跌幅 (%) 股价走势图", fontsize=12, ha='center')
    plt.axis('off')
    plt.title("字体显示测试", fontsize=16)
    plt.tight_layout()
    plt.show()

print("\n" + "="*50)
print("🔧 运行字体测试")
print("="*50)
test_chinese_font()

### 表 3.3

In [ ]:
# 使用 tushare 获取历史股价数据 - 确保导入
import pandas as pd

# 获取美的集团过去一年的日线数据
historical_data = pro.daily(ts_code='000333.SZ', 
                           start_date='20240508', 
                           end_date='20250507')
historical_data = historical_data.sort_values('trade_date')  # 按日期排序
historical_data

In [ ]:
# 获取多只股票的历史数据 - 确保导入
import pandas as pd

# 获取美的集团、平安银行、万科A的历史数据
stock_codes = ['000333.SZ', '000001.SZ', '000002.SZ']
hist_data_2023 = {}

for code in stock_codes:
    df = pro.daily(ts_code=code, start_date='20230101', end_date='20231231')
    hist_data_2023[code] = df.sort_values('trade_date')

# 显示结果
for code, df in hist_data_2023.items():
    print(f"\n股票代码: {code}")
    print(f"数据条数: {len(df)}")
    if not df.empty:
        print(df.head(3))

In [ ]:
# 图 3.3: 绘制多只股票收盘价走势图 - 修复中文字体
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import numpy as np

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 检查系统中可用的字体
# import matplotlib.font_manager as fm
# fonts = [font.name for font in fm.fontManager.ttflist]
# print("可用字体数量:", len(fonts))
# print("是否包含中文字体:", any('SimHei' in font or 'Microsoft' in font for font in fonts))

def plot_closing_prices_tushare(data_dict):
    plt.figure(figsize=(12, 8))
    
    for code, df in data_dict.items():
        if not df.empty:
            # 将日期转换为 datetime
            df['trade_date'] = pd.to_datetime(df['trade_date'], format='%Y%m%d')
            # 计算涨跌幅百分比 (以第一个交易日为基准)
            price_change = (df['close'] / df['close'].iloc[0] * 100)
            plt.plot(df['trade_date'], price_change, label=code)
    
    plt.ylabel("涨跌幅 (%)", fontsize=12)
    plt.title(f"股价走势图 (以首日为基准 100%)", fontsize=15)
    plt.legend(fontsize=10)
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# 绘制 2023 年股价走势
plot_closing_prices_tushare(hist_data_2023)

In [ ]:
# 带不同线型的股价走势图 - 修复中文字体
import matplotlib.pyplot as plt
import pandas as pd

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

def plot_closing_prices_dotted_tushare(data_dict):
    # 线型样式
    line_styles = ['-', ':', '--', '-.']
    
    plt.figure(figsize=(12, 8))
    
    for i, (code, df) in enumerate(data_dict.items()):
        if not df.empty:
            # 将日期转换为 datetime
            df['trade_date'] = pd.to_datetime(df['trade_date'], format='%Y%m%d')
            # 计算涨跌幅百分比 (以第一个交易日为基准)
            price_change = (df['close'] / df['close'].iloc[0] * 100)
            style = line_styles[i % len(line_styles)]  # 循环使用线型
            plt.plot(df['trade_date'], price_change, linestyle=style, label=code)
    
    plt.ylabel("涨跌幅 (%)", fontsize=12)
    plt.title("股价走势图", fontsize=15)
    plt.legend(fontsize=10)
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

plot_closing_prices_dotted_tushare(hist_data_2023)

In [ ]:
# 图 3.4: 2024 年股价走势对比 - 确保导入
import pandas as pd

# 获取 2024 年数据
hist_prices_24 = {}

stock_codes = ['000333.SZ', '000001.SZ', '000002.SZ']
for code in stock_codes:
    df = pro.daily(ts_code=code, start_date='20231122', end_date='20241121')
    hist_prices_24[code] = df.sort_values('trade_date')

plot_closing_prices_dotted_tushare(hist_prices_24)

In [ ]:
# 计算简单收益率 - 确保导入
import pandas as pd

simple_returns = {}
for code, df in hist_prices_24.items():
    if not df.empty:
        # 计算简单收益率
        close_prices = df['close']
        returns = close_prices.pct_change().dropna()
        simple_returns[code] = returns

# 转换为 DataFrame
simple_returns_df = pd.DataFrame(simple_returns)
simple_returns_df.head()

In [ ]:
# 计算对数收益率 - 确保导入
import numpy as np
import pandas as pd

log_returns = {}
for code, df in hist_prices_24.items():
    if not df.empty:
        # 计算对数收益率
        close_prices = df['close']
        log_rets = np.log(close_prices / close_prices.shift()).dropna()
        log_returns[code] = log_rets

# 转换为 DataFrame
log_returns_df = pd.DataFrame(log_returns)
log_returns_df.head()

In [ ]:
# 图 3.5: 对数收益率分布直方图 - 修复中文字体
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 计算 2023 年对数收益率
log_rets_2023 = {}
for code, df in hist_data_2023.items():
    if not df.empty:
        close_prices = df['close']
        log_rets = np.log(close_prices / close_prices.shift()).dropna()
        log_rets_2023[code] = log_rets

# 转换为 DataFrame
log_rets_2023_df = pd.DataFrame(log_rets_2023)

# 绘制直方图
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, (code, returns) in enumerate(log_rets_2023.items()):
    axes[i].hist(returns, bins=35, alpha=0.7)
    axes[i].set_title(f'{code} 对数收益率分布', fontsize=12)
    axes[i].set_xlabel('对数收益率', fontsize=10)
    axes[i].set_ylabel('频数', fontsize=10)
    
plt.tight_layout()
plt.show()

In [ ]:
# 计算收益率统计摘要 - 确保导入
import pandas as pd

summary = log_rets_2023_df.agg(['mean', 'std', 'var']).T
summary.columns = ['均值', '标准差', '方差']
summary

# 商业数据源

### Finviz

In [ ]:
# Finviz 需要付费订阅获取 token
FINVIZ_TOKEN = ""  # 如需获取 token，需要付费订阅
print("注意：Finviz 需要付费订阅才能获取数据访问权限")

In [ ]:
# 使用 Finviz API 导出数据（需要付费订阅）
import requests

if FINVIZ_TOKEN:
    URL = f"https://elite.finviz.com/export.ashx?[allYourFilters]&auth={FINVIZ_TOKEN}"
    response = requests.get(URL)
    with open("finviz_export.csv", "wb") as f:
        f.write(response.content)
    print("数据已导出到 finviz_export.csv")
else:
    print("请设置 FINVIZ_TOKEN 以使用 Finviz API")

### EODHD

In [ ]:
# EODHD 需要订阅获取密钥
# 免费版本提供基本功能，完整功能需要付费订阅
import os

# 从环境变量获取 EODHD 密钥
EOD_SECRET = os.getenv("eod.secret")
if not EOD_SECRET:
    print("请设置环境变量 eod.secret 以使用 EODHD API")
else:
    print("EODHD 密钥已加载")

In [ ]:
# 初始化 EODHD 客户端（需要先安装 eodhd 库）
# pip install eodhd

# 注意：如果没有 EODHD 密钥，这部分代码会报错
try:
    from eodhd import APIClient
    if EOD_SECRET:
        client = APIClient(EOD_SECRET)
        print("EODHD 客户端已初始化")
    else:
        print("请先设置 EODHD 密钥")
except ImportError:
    print("请先安装 eodhd 库：pip install eodhd")
except Exception as e:
    print(f"初始化 EODHD 客户端时出错：{e}")

In [ ]:
# 表 3.6: 获取交易所列表
if 'client' in locals():
    try:
        exchanges = client.get_exchanges()
        exchanges
    except Exception as e:
        print(f"获取交易所列表时出错：{e}")
else:
    print("请先初始化 EODHD 客户端")

In [ ]:
# 表 3.7: 获取股票历史数据（EODHD）
if 'client' in locals():
    try:
        # 获取股票历史数据
        resp = client.get_eod_historical_stock_market_data(
            symbol='RR.LSE', 
            period='d', 
            from_date='2024-06-01', 
            to_date='2024-06-17', 
            order='a'
        )
        eodhd_df = pd.DataFrame(resp)
        eodhd_df
    except Exception as e:
        print(f"获取股票历史数据时出错：{e}")
else:
    print("请先初始化 EODHD 客户端")

### Alpha Vantage

In [ ]:
# Alpha Vantage 需要从环境变量获取密钥
# 从环境变量获取 Alpha Vantage 密钥
ALPHA_VANTAGE_SECRET = os.getenv("datasource.alphavantage.secret")
if not ALPHA_VANTAGE_SECRET:
    print("请设置环境变量 datasource.alphavantage.secret 以使用 Alpha Vantage API")
else:
    print("Alpha Vantage 密钥已加载")

In [ ]:
# 表 3.8: 使用 Alpha Vantage 获取日线数据
if ALPHA_VANTAGE_SECRET:
    try:
        from alpha_vantage.timeseries import TimeSeries
        import pandas as pd
        
        symbol = 'AAPL'
        ts = TimeSeries(ALPHA_VANTAGE_SECRET, output_format='pandas')
        data, meta_data = ts.get_daily(symbol=symbol, outputsize='compact')
        alpha_vantage_df = pd.DataFrame(data)
        alpha_vantage_df
    except Exception as e:
        print(f"获取 Alpha Vantage 数据时出错：{e}")
        print("可能需要付费订阅才能访问完整数据")
else:
    print("请先设置 Alpha Vantage 密钥")

In [ ]:
# 表 3.9: 转置数据以便查看
if 'alpha_vantage_df' in locals():
    alpha_vantage_df.T
else:
    print("请先运行前一个单元格获取数据")

In [ ]:
# 获取完整历史数据（需要付费订阅）
if ALPHA_VANTAGE_SECRET:
    try:
        data, meta_data = ts.get_daily_adjusted(symbol, outputsize='full')
        print("获取完整历史数据成功")
    except Exception as e:
        print(f"获取完整历史数据失败：{e}")
        print("Alpha Vantage 完整数据需要付费订阅")
else:
    print("请先设置 Alpha Vantage 密钥")

In [ ]:
# 获取新闻情感分析 - 确保导入
import requests

if ALPHA_VANTAGE_SECRET:
    try:
        ticker = 'SMR'
        url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={ALPHA_VANTAGE_SECRET}'
        r = requests.get(url)
        data = r.json()
        
        if 'feed' in data:
            news_df = pd.DataFrame.from_dict(data['feed'])
            print(f"成功获取 {len(news_df)} 条新闻")
        else:
            print("未获取到新闻数据")
    except Exception as e:
        print(f"获取新闻情感分析时出错：{e}")
else:
    print("请先设置 Alpha Vantage 密钥")

In [ ]:
# 显示新闻数据 - 确保导入
import pandas as pd

if 'news_df' in locals():
    print(f"新闻数据列：{list(news_df.columns)}")
    print(f"\n数据形状：{news_df.shape}")
    print("\n前几条新闻标题：")
    if 'title' in news_df.columns:
        print(news_df['title'].head())
else:
    print("请先运行前一个单元格获取新闻数据")

### OpenBB

**注意！** 在撰写本书时，OpenBB 不支持最新的 Python 3.13 版本

In [ ]:
# OpenBB 示例代码（需要先安装 openbb 库）
# pip install openbb

try:
    from openbb import obb
    
    # 获取英伟达股价数据
    nvidia_data = obb.equity.price.historical("NVDA").to_df().head()
    print("英伟达股价数据（前5行）：")
    print(nvidia_data)
except ImportError:
    print("请先安装 openbb 库：pip install openbb")
    print("注意：OpenBB 可能不支持 Python 3.13")
except Exception as e:
    print(f"使用 OpenBB 时出错：{e}")
    print("可能需要配置 API 密钥或版本兼容性问题")

In [ ]:
# 使用 yfinance 数据提供商的 OpenBB
try:
    if 'obb' in locals():
        nvidia_yf = obb.equity.price.historical("NVDA", provider="yfinance").to_df().head()
        print("使用 yfinance 提供商的英伟达数据：")
        print(nvidia_yf)
    else:
        print("请先初始化 OpenBB")
except Exception as e:
    print(f"使用 OpenBB yfinance 提供商时出错：{e}")

In [ ]:
# OpenBB 多数据类型示例
try:
    if 'obb' in locals():
        # 货币数据
        usd_gbp = obb.currency.price.historical("USDGBP").to_df().head()
        print("USD/GBP 汇率数据：")
        print(usd_gbp)
        
        # 国债收益率
        treasury = obb.fixedincome.government.treasury_rates(start_date="2024-01-02").to_df().dropna()
        print("\n美国国债收益率：")
        print(treasury.head())
        
        # 加密货币数据
        sol_data = obb.crypto.price.historical("SOLUSD").to_df().tail()
        print("\nSOL/USD 加密货币数据：")
        print(sol_data)
    else:
        print("请先初始化 OpenBB")
except Exception as e:
    print(f"使用 OpenBB 多数据类型时出错：{e}")
    print("可能需要配置相应的 API 密钥")